NEURAL NETWORKS PROJECT
Load Existing Models + Strategic New Experiments

Features:
- Load pre-trained models (safe backup)
- 2-3 new strategic experiments (hidden layers, learning rate)

SETUP:
1. Runtime → Change runtime type → T4 GPU
2. Run all cells
3. Existing models will be loaded, new ones trained

# **LIBRARIES**

In [ ]:
from google.colab import drive
import os
import shutil
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import time
import pickle
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.decomposition import PCA

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Subset
import torchvision
import torchvision.transforms as transforms

# **GOOGLE DRIVE SETUP & BACKUP STRATEGY**

In [ ]:
# Mount Drive
drive.mount('/content/drive', force_remount=True)

# Project folders
PROJECT_ROOT = '/content/drive/MyDrive/NN_CIFAR10_Project'
MODELS_DIR = f'{PROJECT_ROOT}/trained_models'  # Safe backup
RESULTS_DIR = f'{PROJECT_ROOT}/results'
NEW_EXPERIMENTS_DIR = f'{PROJECT_ROOT}/new_experiments'

# Create directories
for dir_path in [PROJECT_ROOT, MODELS_DIR, RESULTS_DIR, NEW_EXPERIMENTS_DIR]:
    os.makedirs(dir_path, exist_ok=True)

print(f"Project root: {PROJECT_ROOT}")
print(f"Existing models: {MODELS_DIR}")
print(f"New experiments: {NEW_EXPERIMENTS_DIR}")

# Backup timestamp
TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
print(f"Session timestamp: {TIMESTAMP}")

Mounted at /content/drive
Project root: /content/drive/MyDrive/NN_CIFAR10_Project
Existing models: /content/drive/MyDrive/NN_CIFAR10_Project/trained_models
New experiments: /content/drive/MyDrive/NN_CIFAR10_Project/new_experiments
Session timestamp: 20251123_065646


# **GPU SETUP**

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n🖥️  Device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    torch.backends.cudnn.benchmark = True
else:
    print("GPU not available! Switch to GPU in Runtime settings.")

# Seeds
torch.manual_seed(42)
np.random.seed(42)



🖥️  Device: cpu
GPU not available! Switch to GPU in Runtime settings.


# **MODEL ARCHITECTURES**

In [ ]:

class BestMLP(nn.Module):
    """MLP architecture (must match existing model)"""
    def __init__(self):
        super(BestMLP, self).__init__()
        self.network = nn.Sequential(
            nn.Flatten(),
            nn.Linear(3072, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 10)
        )
    def forward(self, x):
        return self.network(x)

class BestCNN(nn.Module):
    """CNN architecture (must match existing model)"""
    def __init__(self):
        super(BestCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)

        self.conv5 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.conv6 = nn.Conv2d(256, 256, 3, padding=1)
        self.bn6 = nn.BatchNorm2d(256)

        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)
        x = F.dropout2d(x, 0.1, training=self.training)

        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, 2)
        x = F.dropout2d(x, 0.2, training=self.training)

        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = F.max_pool2d(x, 2)
        x = F.dropout2d(x, 0.3, training=self.training)

        return self.classifier(x)

# NEW: Configurable MLP for experiments
class ConfigurableMLP(nn.Module):
    """MLP with configurable hidden layers"""
    def __init__(self, hidden_sizes=[512], dropout=0.3):
        super(ConfigurableMLP, self).__init__()
        layers = [nn.Flatten()]

        in_size = 3072
        for h_size in hidden_sizes:
            layers.extend([
                nn.Linear(in_size, h_size),
                nn.BatchNorm1d(h_size),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            in_size = h_size

        layers.append(nn.Linear(in_size, 10))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# **LOAD EXISTING MODELS & RESULTS**

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

existing_results = {}
loaded_models = {}

# Try to load existing models
existing_model_files = {
    'MLP-Best': 'MLP-Best_model.pth',
    'CNN-Best-100ep': 'CNN-Best-100ep_model.pth',
    'CNN-Best-150ep': 'CNN-Best-150ep_model.pth'
}

for name, filename in existing_model_files.items():
    model_path = f'{PROJECT_ROOT}/{filename}'

    if os.path.exists(model_path):
        try:
            if 'MLP' in name:
                model = BestMLP().to(device)
            else:
                model = BestCNN().to(device)

            model.load_state_dict(torch.load(model_path, map_location=device))
            model.eval()

            loaded_models[name] = model
            print(f"Loaded: {name}")
        except Exception as e:
            print(f"Failed to load {name}: {e}")
    else:
        print(f"Not found: {name} (will need to train)")

# Try to load existing results
results_file = f'{PROJECT_ROOT}/complete_results.pkl'
if os.path.exists(results_file):
    try:
        with open(results_file, 'rb') as f:
            saved_data = pickle.load(f)
            existing_results = saved_data.get('nn_results', {})
        print(f"\nLoaded existing results: {len(existing_results)} experiments")
    except:
        print("\nCould not load existing results")

# Load baseline results
baseline_results = {}
baseline_file = f'{PROJECT_ROOT}/baseline_results.pkl'
if os.path.exists(baseline_file):
    with open(baseline_file, 'rb') as f:
        baseline_results = pickle.load(f)
    print(f"Loaded baseline results")
else:
    print("Baseline results not found - will compute")

print(f"\nStatus:")
print(f"Loaded models: {len(loaded_models)}")
print(f"Existing results: {len(existing_results)}")
print(f"Baseline results: {len(baseline_results)}")

Loaded: MLP-Best
Loaded: CNN-Best-100ep
Loaded: CNN-Best-150ep

Could not load existing results
Baseline results not found - will compute

Status:
Loaded models: 3
Existing results: 0
Baseline results: 0


# **DATA LOADING**

In [ ]:
# Transforms
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Load CIFAR-10
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_size = int(0.9 * len(trainset))
val_size = len(trainset) - train_size
trainset, valset = random_split(trainset, [train_size, val_size])

print(f"Train: {train_size:,} | Val: {val_size:,} | Test: {len(testset):,}")


Train: 45,000 | Val: 5,000 | Test: 10,000


# **TRAINING UTILITIES**

In [ ]:
class QuickTrainer:
    """Fast trainer for experiments"""
    def __init__(self, model, trainloader, valloader, criterion, optimizer, device, scheduler=None):
        self.model = model.to(device)
        self.trainloader = trainloader
        self.valloader = valloader
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        self.history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
        self.best_val_acc = 0.0
        self.best_model_state = None

    def train_epoch(self):
        self.model.train()
        running_loss, correct, total = 0.0, 0, 0
        for inputs, labels in self.trainloader:
            inputs, labels = inputs.to(self.device), labels.to(self.device)
            self.optimizer.zero_grad()
            outputs = self.model(inputs)
            loss = self.criterion(outputs, labels)
            loss.backward()
            self.optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        return running_loss / total, 100. * correct / total

    def validate(self):
        self.model.eval()
        running_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in self.valloader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                running_loss += loss.item() * inputs.size(0)
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
        return running_loss / total, 100. * correct / total

    def train(self, epochs):
        print(f"Training for {epochs} epochs...")
        start_time = time.time()

        for epoch in range(epochs):
            train_loss, train_acc = self.train_epoch()
            val_loss, val_acc = self.validate()

            if self.scheduler:
                self.scheduler.step()

            self.history['train_loss'].append(train_loss)
            self.history['train_acc'].append(train_acc)
            self.history['val_loss'].append(val_loss)
            self.history['val_acc'].append(val_acc)

            if val_acc > self.best_val_acc:
                self.best_val_acc = val_acc
                self.best_model_state = self.model.state_dict().copy()

            if (epoch + 1) % 10 == 0 or epoch == 0:
                print(f"  Epoch {epoch+1:3d}/{epochs} | TL: {train_loss:.4f} TA: {train_acc:5.2f}% | VL: {val_loss:.4f} VA: {val_acc:5.2f}%")

        self.model.load_state_dict(self.best_model_state)
        elapsed = time.time() - start_time
        print(f"Completed in {elapsed:.1f}s | Best Val: {self.best_val_acc:.2f}%")
        return self.history, elapsed

def evaluate_model(model, dataloader, device):
    """Evaluate model and return detailed results"""
    model.eval()
    all_preds, all_labels, all_probs = [], [], []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            probs = F.softmax(outputs, dim=1)
            _, predicted = outputs.max(1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.numpy())
            all_probs.extend(probs.cpu().numpy())

    accuracy = 100. * np.mean(np.array(all_preds) == np.array(all_labels))
    cm = confusion_matrix(all_labels, all_preds)

    return {
        'accuracy': accuracy,
        'predictions': np.array(all_preds),
        'labels': np.array(all_labels),
        'probabilities': np.array(all_probs),
        'confusion_matrix': cm
    }

# **COMPUTE BASELINES**

In [ ]:
if not baseline_results:
    # Prepare data for baselines
    transform_basic = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])

    trainset_basic = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_basic)
    testset_basic = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_basic)

    def prepare_data(dataset, max_samples=2000):
        loader = DataLoader(dataset, batch_size=1000, shuffle=False)
        X_list, y_list = [], []
        for images, labels in loader:
            X_list.append(images.numpy())
            y_list.append(labels.numpy())
            if max_samples and len(y_list) * 1000 >= max_samples:
                break
        X = np.vstack(X_list).reshape(len(np.hstack(y_list)), -1)
        y = np.hstack(y_list)
        return X, y

    X_train, y_train = prepare_data(trainset_basic, max_samples=None)
    X_test, y_test = prepare_data(testset_basic, max_samples=2000)

    # PCA
    print("\nApplying PCA-25...")
    pca = PCA(n_components=25, random_state=42)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    explained_var = pca.explained_variance_ratio_.sum() * 100
    print(f"Explained variance: {explained_var:.2f}%")


    # 1-NN
    print("\nRunning 1-NN...")
    start = time.time()
    predictions_1nn = []
    for i, x in enumerate(X_test_pca):
        if (i + 1) % 500 == 0:
            print(f"   {i+1}/{len(X_test_pca)}")
        distances = np.sqrt(np.sum((X_train_pca - x) ** 2, axis=1))
        k_idx = np.argsort(distances)[:3]
        predictions_1nn.append(Counter(y_train[k_idx]).most_common(1)[0][0])
    acc_1nn = 100. * np.mean(np.array(predictions_1nn) == y_test)
    time_1nn = time.time() - start
    print(f"1-NN: {acc_1nn:.2f}% ({time_1nn:.1f}s)")

    # 3-NN
    print("\nRunning 3-NN...")
    start = time.time()
    predictions_3nn = []
    for i, x in enumerate(X_test_pca):
        if (i + 1) % 500 == 0:
            print(f"   {i+1}/{len(X_test_pca)}")
        distances = np.sqrt(np.sum((X_train_pca - x) ** 2, axis=1))
        k_idx = np.argsort(distances)[:3]
        predictions_3nn.append(Counter(y_train[k_idx]).most_common(1)[0][0])
    acc_3nn = 100. * np.mean(np.array(predictions_3nn) == y_test)
    time_3nn = time.time() - start
    print(f"3-NN: {acc_3nn:.2f}% ({time_3nn:.1f}s)")

    # NCC
    print("\nRunning NCC...")
    start = time.time()
    centroids = np.array([X_train_pca[y_train == c].mean(axis=0) for c in range(10)])
    predictions_ncc = [np.argmin(np.sqrt(np.sum((centroids - x) ** 2, axis=1))) for x in X_test_pca]
    acc_ncc = 100. * np.mean(np.array(predictions_ncc) == y_test)
    time_ncc = time.time() - start
    print(f"NCC: {acc_ncc:.2f}% ({time_ncc:.1f}s)")

    baseline_results = {
        '1-NN (PCA-25)': {'accuracy': acc_1nn, 'time': time_1nn, 'predictions': predictions_1nn},
        '3-NN (PCA-25)': {'accuracy': acc_3nn, 'time': time_3nn, 'predictions': predictions_3nn},
        'NCC (PCA-25)': {'accuracy': acc_ncc, 'time': time_ncc, 'predictions': predictions_ncc}
    }

    # Save
    with open(f'{RESULTS_DIR}/baseline_results.pkl', 'wb') as f:
        pickle.dump(baseline_results, f)
    print(f"\nBaselines saved!")


Applying PCA-25...
Explained variance: 77.11%

Running 1-NN...
   500/2000
   1000/2000
   1500/2000
   2000/2000
1-NN: 40.05% (16.1s)

Running 3-NN...
   500/2000
   1000/2000
   1500/2000
   2000/2000
3-NN: 40.05% (14.7s)

Running NCC...
NCC: 27.90% (0.1s)

Baselines saved!


# **NEW EXPERIMENTS**

In [ ]:
new_experiments = {
    # Experiment 1: MLP with different hidden layers (~30 min)
    'MLP-Small-256': {
        'model_class': ConfigurableMLP,
        'model_args': {'hidden_sizes': [256], 'dropout': 0.3},
        'epochs': 50,
        'batch_size': 128,
        'lr': 0.001,
        'optimizer': 'AdamW'
    },

    # Experiment 2: MLP with larger architecture (~35 min)
    'MLP-Large-1024-512': {
        'model_class': ConfigurableMLP,
        'model_args': {'hidden_sizes': [1024, 512], 'dropout': 0.3},
        'epochs': 50,
        'batch_size': 128,
        'lr': 0.001,
        'optimizer': 'AdamW'
    },

    # Experiment 3: MLP with different learning rate (~30 min)
    'MLP-512-LR001': {
        'model_class': ConfigurableMLP,
        'model_args': {'hidden_sizes': [512], 'dropout': 0.3},
        'epochs': 50,
        'batch_size': 128,
        'lr': 0.01,  # Higher LR
        'optimizer': 'AdamW'
    },
}

print(f"\nPlanned experiments: {len(new_experiments)}")
print(f"Estimated time: ~90-120 minutes total")
print("\nExperiments:")
for i, name in enumerate(new_experiments.keys(), 1):
    print(f"   {i}. {name}")

new_results = {}

for exp_name, config in new_experiments.items():
    print(f"\n{'━'*70}")
    print(f"EXPERIMENT: {exp_name}")
    print(f"{'━'*70}")

    # Check if already exists
    model_file = f'{NEW_EXPERIMENTS_DIR}/{exp_name}_model.pth'
    if os.path.exists(model_file):
        print(f"Already trained! Loading from cache...")
        model = config['model_class'](**config['model_args']).to(device)
        model.load_state_dict(torch.load(model_file))

        # Load results
        results_file = f'{NEW_EXPERIMENTS_DIR}/{exp_name}_results.pkl'
        if os.path.exists(results_file):
            with open(results_file, 'rb') as f:
                new_results[exp_name] = pickle.load(f)
            print(f"Loaded cached results")
            continue

    # Create model
    model = config['model_class'](**config['model_args']).to(device)
    params = sum(p.numel() for p in model.parameters())
    print(f"Parameters: {params:,}")

    # Data loaders
    trainloader = DataLoader(trainset, batch_size=config['batch_size'], shuffle=True, num_workers=2, pin_memory=True)
    valloader = DataLoader(valset, batch_size=config['batch_size'], shuffle=False, num_workers=2, pin_memory=True)
    testloader = DataLoader(testset, batch_size=config['batch_size'], shuffle=False, num_workers=2, pin_memory=True)

    # Training setup
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

    if config['optimizer'] == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=config['lr'], momentum=0.9, weight_decay=5e-4)
    else:
        optimizer = optim.AdamW(model.parameters(), lr=config['lr'], weight_decay=0.01)

    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config['epochs'])

    # Train
    trainer = QuickTrainer(model, trainloader, valloader, criterion, optimizer, device, scheduler)
    history, train_time = trainer.train(epochs=config['epochs'])

    # Evaluate
    print(f"\nEvaluating on test set...")
    test_results = evaluate_model(model, testloader, device)

    # Store results
    new_results[exp_name] = {
        'model': model,
        'history': history,
        'test_accuracy': test_results['accuracy'],
        'best_val_accuracy': trainer.best_val_acc,
        'training_time': train_time,
        'test_results': test_results,
        'config': config
    }

    print(f"Test Accuracy: {test_results['accuracy']:.2f}%")

    # Save model & results
    torch.save(model.state_dict(), model_file)
    with open(f'{NEW_EXPERIMENTS_DIR}/{exp_name}_results.pkl', 'wb') as f:
        pickle.dump(new_results[exp_name], f)
    print(f"Saved: {exp_name}")


Planned experiments: 3
Estimated time: ~90-120 minutes total

Experiments:
   1. MLP-Small-256
   2. MLP-Large-1024-512
   3. MLP-512-LR001

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
EXPERIMENT: MLP-Small-256
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Parameters: 789,770
Training for 50 epochs...


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  Epoch   1/50 | TL: 1.9966 TA: 31.07% | VL: 1.8872 VA: 36.06%
  Epoch  10/50 | TL: 1.7934 TA: 41.46% | VL: 1.7219 VA: 45.12%
  Epoch  20/50 | TL: 1.7502 TA: 44.09% | VL: 1.6618 VA: 48.74%
  Epoch  30/50 | TL: 1.7169 TA: 45.59% | VL: 1.6504 VA: 49.02%
  Epoch  40/50 | TL: 1.6992 TA: 46.47% | VL: 1.6275 VA: 50.06%
  Epoch  50/50 | TL: 1.6966 TA: 46.76% | VL: 1.6174 VA: 50.46%
Completed in 2400.0s | Best Val: 50.76%

Evaluating on test set...
Test Accuracy: 52.52%
💾 Saved: MLP-Small-256

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
EXPERIMENT: MLP-Large-1024-512
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Parameters: 3,679,754
Training for 50 epochs...


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  Epoch   1/50 | TL: 1.9774 TA: 31.53% | VL: 1.8248 VA: 38.82%
  Epoch  10/50 | TL: 1.6917 TA: 44.98% | VL: 1.6325 VA: 47.48%
  Epoch  20/50 | TL: 1.6214 TA: 48.82% | VL: 1.5616 VA: 51.56%
  Epoch  30/50 | TL: 1.5695 TA: 51.54% | VL: 1.5072 VA: 54.46%
  Epoch  40/50 | TL: 1.5314 TA: 53.32% | VL: 1.4756 VA: 56.34%
  Epoch  50/50 | TL: 1.5184 TA: 53.99% | VL: 1.4626 VA: 56.24%
Completed in 3257.3s | Best Val: 56.44%

Evaluating on test set...
Test Accuracy: 58.05%
💾 Saved: MLP-Large-1024-512

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
EXPERIMENT: MLP-512-LR001
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Parameters: 1,579,530
Training for 50 epochs...


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  Epoch   1/50 | TL: 2.0454 TA: 29.60% | VL: 1.8744 VA: 36.86%
  Epoch  10/50 | TL: 1.8050 TA: 41.01% | VL: 1.7325 VA: 44.06%
  Epoch  20/50 | TL: 1.7585 TA: 43.20% | VL: 1.6804 VA: 46.40%
  Epoch  30/50 | TL: 1.7165 TA: 45.36% | VL: 1.6401 VA: 48.66%
  Epoch  40/50 | TL: 1.6830 TA: 47.11% | VL: 1.5948 VA: 50.74%
  Epoch  50/50 | TL: 1.6628 TA: 48.28% | VL: 1.5911 VA: 51.56%
Completed in 2692.0s | Best Val: 53.14%

Evaluating on test set...
Test Accuracy: 54.44%
💾 Saved: MLP-512-LR001
